In [1]:
import numpy as np
import keras
import keras.backend as K
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from PIL import Image
import random
import math
import matplotlib.pyplot as plt
import dendropy
import csv
from dendropy.calculate import treecompare
import sys
import numpy
import pickle
from sklearn.model_selection import train_test_split
from keras.optimizers import RMSprop
numpy.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
sys.path.append("../..")

In [4]:
import Bandelt_Encode_Decode.Bandelt_Encode_func as BN_Encode_func
import Bandelt_Encode_Decode.Bandelt_Decode_func as BN_Decode_func
import Bandelt_Encode_Decode.Bandelt_Node as BN

# Loading Data

In [8]:
X_Input_Alignment_Data = np.load('../../ALL_DATA/499_dataset_Input_Output/X_Input_Alignment_Data.npy')

In [9]:
print(X_Input_Alignment_Data.shape)

(499, 100, 1000, 5)


In [13]:
train_X,valid_X,train_ground,valid_ground = train_test_split(X_Input_Alignment_Data,
                                                             X_Input_Alignment_Data, 
                                                             test_size=0.2, 
                                                             random_state=13)

## Shuffle Input Data

In [10]:
for i in range(X_Input_Alignment_Data.shape[0]):
    shuffle_indices_100 = random.sample(range(0, 100), 100)
    shuffle_indices_1000 = random.sample(range(0, 1000), 1000)
    tmp = X_Input_Alignment_Data[i][shuffle_indices_100]
    tmp = tmp[:, shuffle_indices_1000]
    X_Input_Alignment_Data[i] = tmp

In [12]:
for index, sample in enumerate(X_Input_Alignment_Data[0]):
    print('Sample ', index, ': ', )
    for sequence in sample:
#         print("A: ", sequence.astype(int))
#         print("B: ", np.array([0,0,0,0,0]))
        if np.array_equal(sequence.astype(int), np.array([0,0,0,0,0])):
            print('_', end='')
        elif np.array_equal(sequence.astype(int), np.array([1,0,0,0,0])):
            print('A', end='')
        elif np.array_equal(sequence.astype(int), np.array([0,1,0,0,0])):
            print('T', end='')
        elif np.array_equal(sequence.astype(int), np.array([0,0,1,0,0])):
            print('C', end='')
        elif np.array_equal(sequence.astype(int), np.array([0,0,0,1,0])):
            print('G', end='')
        elif np.array_equal(sequence.astype(int), np.array([0,0,0,0,1])):
            print('X', end='')
    print('\n')

Sample  0 : 
___________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

________________CG__________________G____C______T_______C__________C____G_C__C__G_____C_________C____________C___________________________G_______________________________CG________C___G____________X________CA_G________________________G________TC___T__C________C____GC___________T__CT_________________________CGG__________G___________T___C________T________CG____C_G___________G___G___C_______________C____________AC__________C________________________________________C____________________________G__C_____G___________C________________G_________________________T____C____________CG____C_C_______C___________________________________________C__________C__________X______________X____________________C___C_________________________C_GG____C________T____G________________________C_________________________C_A____A_G________C_______T___________C___G_____G____C_____C____________T_G____G___G________________AC____________________C____C________G_____G___C______________C___________T_____________________A________

________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

________________AG__________________G____C______T_______C__________T____G_T__C__G_____C_________G____________C___________________________G_______________________________CG________C___G____________G________CT_G________________________G________TC___T__C________T____AC___________T__CT_________________________CGG__________A___________T___C________T________CG____C_G___________G___G___C_______________T____________AC__________C________________________________________C____________________________G__C_____G___________T________________G_________________________G____C____________CA____C_C_______C___________________________________________C__________C__________C______________C____________________T___A_________________________C_GG____C________T____G________________________C_________________________C_C____C_G________T_______T___________C___C_____G____T_____C____________T_G____A___G________________AC____________________C____C________G_____T___C______________C___________A_____________________C________

...

________________XG__________________G____C______T_______C__________C____G_T__C__G_____C_________C____________C___________________________G_______________________________CG________C___G____________G________CA_G________________________G________TC___T__C________A____AC___________T__CT_________________________CGG__________A___________T___C________T________CG____C_G___________G___G___C_______________C____________AC__________C________________________________________C____________________________G__C_____G___________C________________X_________________________G____C____________CA____C_C_______C___________________________________________C__________C__________C______________C____________________T___A_________________________C_GG____C________T____G________________________C_________________________C_C____T_G________T_______T___________C___C_____G____T_____C____________T_G____A___G________________AC____________________C____C________G_____T___C______________C___________A_____________________C________

________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

________________AG__________________G____C______T_______C__________C____G_T__C__G_____C_________C____________C___________________________G_______________________________CG________C___G____________G________CA_G________________________G________TC___T__C________A____AC___________T__CT_________________________CGG__________A___________T___C________T________CG____C_G___________G___G___C_______________C____________AC__________C________________________________________C____________________________G__C_____G___________C________________G_________________________G____C____________CA____C_C_______C___________________________________________C__________C__________C______________C____________________T___A_________________________C_GG____C________T____G________________________C_________________________C_C____T_G________T_______T___________C___C_____G____T_____C____________T_G____A___G________________AC____________________C____C________G_____X___C______________C___________A_____________________C________

________________AG__________________G____G______T_______C__________C____G_T__T__G_____C_________C____________C___________________________G_______________________________CG________C___G____________G________CC_G________________________G________TC___T__C________T____AC___________G__CT_________________________CGG__________A___________T___C________A________CG____C_G___________G___G___C_______________C____________AT__________C________________________________________C____________________________G__C_____A___________C________________A_________________________T____G____________CA____C_C_______C___________________________________________C__________C__________C______________C____________________C___C_________________________C_GA____C________C____G________________________C_________________________C_C____C_G________T_______T___________A___C_____G____C_____C____________T_G____A___G________________AC____________________C____C________G_____G___T______________C___________G_____________________T________

________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [16]:
autoencoder = keras.models.load_model('./output/autoencoder_model.h5')

In [17]:
pred = autoencoder.predict(valid_X)

In [20]:
valid_X.shape

(100, 100, 1000, 5)

In [ ]:
for index, sample in enumerate(X_Input_Alignment_Data[0]):
    print('Sample ', index, ': ', )
    for sequence in sample:
#         print("A: ", sequence.astype(int))
#         print("B: ", np.array([0,0,0,0,0]))
        if np.array_equal(sequence.astype(int), np.array([0,0,0,0,0])):
            print('_', end='')
        elif np.array_equal(sequence.astype(int), np.array([1,0,0,0,0])):
            print('A', end='')
        elif np.array_equal(sequence.astype(int), np.array([0,1,0,0,0])):
            print('T', end='')
        elif np.array_equal(sequence.astype(int), np.array([0,0,1,0,0])):
            print('C', end='')
        elif` np.array_equal(sequence.astype(int), np.array([0,0,0,1,0])):
            print('G', end='')
        elif np.array_equal(sequence.astype(int), np.array([0,0,0,0,1])):
            print('X', end='')
    print('\n')

In [30]:
for valid_X_1_sample, pred_1_sample in zip(valid_X[0], pred[0]):
    for valid_X_1_sequence, pred_1_sequence in zip(valid_X_1_sample, pred_1_sample):
        if not np.array_equal(valid_X_1_sequence.astype(int), pred_1_sequence.astype(int)):
            print('X', end=' ')
            print('(', end='')
            if np.array_equal(valid_X_1_sequence.astype(int), np.array([0,0,0,0,0])):
                print('_', end='')
            elif np.array_equal(valid_X_1_sequence.astype(int), np.array([1,0,0,0,0])):
                print('A', end='')
            elif np.array_equal(valid_X_1_sequence.astype(int), np.array([0,1,0,0,0])):
                print('T', end='')
            elif np.array_equal(valid_X_1_sequence.astype(int), np.array([0,0,1,0,0])):
                print('C', end='')
            elif np.array_equal(valid_X_1_sequence.astype(int), np.array([0,0,0,1,0])):
                print('G', end='')
            elif np.array_equal(valid_X_1_sequence.astype(int), np.array([0,0,0,0,1])):
                print('X', end='')  
            print(',', end='')
            if np.array_equal(pred_1_sequence.astype(int), np.array([0,0,0,0,0])):
                print('_', end='')
            elif np.array_equal(pred_1_sequence.astype(int), np.array([1,0,0,0,0])):
                print('A', end='')
            elif np.array_equal(pred_1_sequence.astype(int), np.array([0,1,0,0,0])):
                print('T', end='')
            elif np.array_equal(pred_1_sequence.astype(int), np.array([0,0,1,0,0])):
                print('C', end='')
            elif np.array_equal(pred_1_sequence.astype(int), np.array([0,0,0,1,0])):
                print('G', end='')
            elif np.array_equal(pred_1_sequence.astype(int), np.array([0,0,0,0,1])):
                print('X', end='')  
            print(')', end='')

    print("")


X (C,_)X (G,_)X (G,_)X (G,_)X (T,_)X (C,_)X (C,_)X (G,_)X (T,_)X (C,_)X (G,_)X (C,_)X (T,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (G,_)X (C,_)X (C,_)X (G,_)X (G,_)X (T,_)X (C,_)X (T,_)X (C,_)X (C,_)X (G,_)X (C,_)X (T,_)X (C,_)X (T,_)X (C,_)X (G,_)X (G,_)X (G,_)X (C,_)X (C,_)X (T,_)X (C,_)X (G,_)X (C,_)X (G,_)X (G,_)X (G,_)X (C,_)X (C,_)X (A,_)X (C,_)X (C,_)X (C,_)X (G,_)X (T,_)X (A,_)X (C,_)X (G,_)X (T,_)X (C,_)X (C,_)X (G,_)X (C,_)X (C,_)X (T,_)X (C,_)X (C,_)X (C,_)X (C,_)X (G,_)X (C,_)X (C,_)X (G,_)X (G,_)X (C,_)X (T,_)X (G,_)X (C,_)X (C,_)X (C,_)X (G,_)X (G,_)X (T,_)X (T,_)X (C,_)X (C,_)X (G,_)X (T,_)X (C,_)X (T,_)X (G,_)X (G,_)X (G,_)X (G,_)X (C,_)X (C,_)X (C,_)X (G,_)X (G,_)X (C,_)X (T,_)X (G,_)X (C,_)
X (A,_)X (G,_)X (G,_)X (C,_)X (T,_)X (C,_)X (C,_)X (G,_)X (T,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (A,_)X (C,_)X (T,_)X (G,_)X (G,_)X (T,_)X (C,_)X (C,_)X (C,_)X (C,_)X (A,_)X (C,_)X (T,_)X (C,_)X (T,_)X (C,_)X (G,_)X (G,_)X (A,_)X (C,_)X (C

X (A,_)X (G,_)X (G,_)X (C,_)X (T,_)X (C,_)X (C,_)X (G,_)X (T,_)X (C,_)X (G,_)X (C,_)X (T,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (A,_)X (C,_)X (A,_)X (G,_)X (G,_)X (T,_)X (C,_)X (T,_)X (C,_)X (T,_)X (A,_)X (A,_)X (T,_)X (C,_)X (T,_)X (C,_)X (G,_)X (G,_)X (A,_)X (T,_)X (C,_)X (T,_)X (C,_)X (G,_)X (C,_)X (G,_)X (G,_)X (G,_)X (C,_)X (C,_)X (A,_)X (C,_)X (C,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (G,_)X (C,_)X (C,_)X (G,_)X (C,_)X (C,_)X (C,_)X (C,_)X (C,_)X (C,_)X (C,_)X (T,_)X (A,_)X (C,_)X (A,_)X (G,_)X (C,_)X (T,_)X (G,_)X (C,_)X (C,_)X (C,_)X (C,_)X (G,_)X (T,_)X (T,_)X (C,_)X (C,_)X (G,_)X (T,_)X (C,_)X (T,_)X (G,_)X (A,_)X (G,_)X (A,_)X (C,_)X (C,_)X (C,_)X (G,_)X (T,_)X (C,_)X (C,_)X (X,_)X (C,_)


X (A,_)X (G,_)X (G,_)X (C,_)X (T,_)X (C,_)X (T,_)X (G,_)X (T,_)X (C,_)X (G,_)X (C,_)X (T,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (A,_)X (C,_)X (A,_)X (G,_)X (G,_)X (T,_)X (C,_)X (T,_)X (C,_)X (C,_)X (A,_)X (A,_)X (T,_)X (C,_)X (T,_)X (C,_)X (A,_)X (G,_)X (A,_)X (T,_)X (

X (T,_)X (C,_)X (G,_)X (G,_)X (C,_)X (C,_)X (C,_)X (G,_)X (T,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (A,_)X (C,_)X (C,_)X (A,_)X (G,_)X (T,_)X (C,_)X (T,_)X (C,_)X (C,_)X (A,_)X (G,_)X (T,_)X (G,_)X (T,_)X (C,_)X (G,_)X (G,_)X (A,_)X (C,_)X (C,_)X (T,_)X (C,_)X (G,_)X (C,_)X (G,_)X (G,_)X (G,_)X (C,_)X (C,_)X (G,_)X (C,_)X (C,_)X (C,_)X (G,_)X (C,_)X (A,_)X (C,_)X (A,_)X (T,_)X (C,_)X (C,_)X (G,_)X (C,_)X (C,_)X (T,_)X (C,_)X (C,_)X (G,_)X (C,_)X (C,_)X (C,_)X (C,_)X (G,_)X (G,_)X (C,_)X (T,_)X (G,_)X (C,_)X (C,_)X (C,_)X (C,_)X (A,_)X (T,_)X (T,_)X (A,_)X (C,_)X (A,_)X (C,_)X (C,_)X (T,_)X (G,_)X (G,_)X (G,_)X (A,_)X (C,_)X (C,_)X (C,_)X (G,_)X (G,_)X (C,_)X (C,_)X (C,_)X (C,_)


X (A,_)X (A,_)X (G,_)X (C,_)X (T,_)X (C,_)X (T,_)X (G,_)X (T,_)X (C,_)X (G,_)X (C,_)X (X,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (G,_)X (X,_)X (A,_)X (G,_)X (G,_)X (T,_)X (C,_)X (T,_)X (C,_)X (T,_)X (A,_)X (C,_)X (T,_)X (C,_)X (T,_)X (C,_)X (G,_)X (G,_)X (A,_)X (T,_)X (

X (A,_)X (G,_)X (G,_)X (C,_)X (T,_)X (C,_)X (T,_)X (G,_)X (T,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (G,_)X (C,_)X (A,_)X (G,_)X (G,_)X (T,_)X (C,_)X (T,_)X (C,_)X (T,_)X (A,_)X (C,_)X (T,_)X (C,_)X (T,_)X (C,_)X (G,_)X (G,_)X (A,_)X (T,_)X (C,_)X (T,_)X (C,_)X (G,_)X (C,_)X (G,_)X (G,_)X (G,_)X (C,_)X (C,_)X (A,_)X (C,_)X (C,_)X (C,_)X (A,_)X (T,_)X (A,_)X (C,_)X (G,_)X (G,_)X (T,_)X (C,_)X (G,_)X (C,_)X (C,_)X (C,_)X (C,_)X (C,_)X (C,_)X (C,_)X (C,_)X (A,_)X (C,_)X (G,_)X (G,_)X (C,_)X (T,_)X (G,_)X (C,_)X (C,_)X (C,_)X (T,_)X (G,_)X (T,_)X (T,_)X (C,_)X (C,_)X (G,_)X (T,_)X (C,_)X (T,_)X (G,_)X (A,_)X (G,_)X (A,_)X (C,_)X (C,_)X (C,_)X (G,_)X (T,_)X (C,_)X (C,_)X (A,_)X (C,_)
X (T,_)X (C,_)X (G,_)X (G,_)X (T,_)X (C,_)X (C,_)X (G,_)X (T,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (C,_)X (G,_)X (A,_)X (T,_)X (G,_)X (A,_)X (G,_)X (T,_)X (C,_)X (T,_)X (C,_)X (T,_)X (A,_)X (G,_)X (T,_)X (G,_)X (T,_)X (C,_)X (G,_)X (G,_)X (G,_)X (T,_)X (C,

## Parameters Initialization

In [9]:
batch_size = 128
epochs = 50
inChannel = 5
BRANCH_NUM = 100
SEQUENCE_LEN = 1000
input_img = Input(shape = (BRANCH_NUM, SEQUENCE_LEN, inChannel))

In [10]:
X_Input_Alignment_Data[0][0]

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0.

In [11]:
def autoencoder(input_img):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #100 x 1000 x 32
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #50 x 500 x 64
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #25 x 250 x 128 (small and thick)

    #decoder
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3) #25 x 250 x 128
    up1 = UpSampling2D((2,2))(conv4) # 14 x 14 x 128
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1) # 50 x 500 x 64
    up2 = UpSampling2D((2,2))(conv5) # 28 x 28 x 64
    decoded = Conv2D(5, (3, 3), activation='sigmoid', padding='same')(up2) # 100 x 1000 x 1
    return decoded

In [12]:
autoencoder = Model(input_img, autoencoder(input_img))
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())

In [13]:
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 1000, 5)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 1000, 32)     1472      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 500, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 500, 64)       18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 250, 64)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 250, 128)      73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 250, 128)      1475

In [14]:
autoencoder_train = autoencoder.fit(train_X, train_ground, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_ground))

Train on 399 samples, validate on 100 samples
Epoch 1/50
399/399 [==============================] - 125s 313ms/step - loss: 0.1949 - val_loss: 0.0188
Epoch 2/50
399/399 [==============================] - 121s 303ms/step - loss: 0.0192 - val_loss: 0.0182
Epoch 3/50


KeyboardInterrupt: 

## Start Building Model

### Load Data